# Recommending Music to train to using an existing Spotify playlist

*Code by Sean Brockway*

### Necessary imports for Spotify and MongoDB Connections

In [1]:
import requests
import json
import pymongo
import spotipy
import spotipy.util as util 
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from spotipy.oauth2 import SpotifyClientCredentials
from pymongo import MongoClient
import urllib.parse

### Setting up connections for MongoDB and Spotify API

In [2]:
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

conn = MongoClient("mongodb://localhost:27017/")
db = conn['gym-music-database']
test_playlist_col = db['Murph Playlist 1']
playlist_songs = test_playlist_col.find({})

**Verifying our data is correct, this is an example of a song in our playlist**

In [3]:
print(playlist_songs[0])

{'_id': ObjectId('5eaad3d105bbe512a38baa77'), 'track': 'Family Affair - Single Version', 'track_id': '2GVEs1sn5Q1jB1976vQoAX', 'artist_name': ['Sly & The Family Stone'], 'artist_id': ['5m8H6zSadhu1j9Yi04VLqD'], 'genres': ['classic rock', 'classic soul', 'disco', 'folk rock', 'funk', 'funk rock', 'mellow gold', 'motown', 'rock', 'roots rock', 'soul', 'southern soul'], 'danceability': 0.819, 'energy': 0.605, 'key': 2, 'loudness': -13.83, 'mode': 0, 'speechiness': 0.132, 'acousticness': 0.669, 'instrumentalness': 0.0163, 'liveness': 0.199, 'valence': 0.797, 'tempo': 108.892, 'duration_ms': 185533, 'time_signature': 4}


**Next we need to make a list of every song's genres**

In [4]:
genre_list = []
for song in playlist_songs:
    genre_list.extend(song['genres'])
genre_list

['classic rock',
 'classic soul',
 'disco',
 'folk rock',
 'funk',
 'funk rock',
 'mellow gold',
 'motown',
 'rock',
 'roots rock',
 'soul',
 'southern soul',
 'funk',
 'motown',
 'quiet storm',
 'soul',
 'funk',
 'motown',
 'quiet storm',
 'soul',
 'funk',
 'motown',
 'quiet storm',
 'soul',
 'classic soul',
 'disco',
 'funk',
 'motown',
 'philly soul',
 'quiet storm',
 'soft rock',
 'soul',
 'southern soul',
 'alternative rock',
 'garage rock',
 'modern rock',
 'permanent wave',
 'rock',
 'deep soft rock',
 'jazz funk',
 'yacht rock',
 'dance pop',
 'aussietronica',
 'indie soul',
 'indie pop',
 'indie soul',
 'british jazz',
 'indie jazz',
 'indie soul',
 'alternative hip hop',
 'hip hop',
 'rap',
 'southern hip hop',
 'underground hip hop',
 'alternative r&b',
 'indie jazz',
 'indie r&b',
 'indie soul',
 'neo r&b',
 'neo soul',
 'soul',
 'indie soul',
 'atl hip hop',
 'hip hop',
 'pop rap',
 'rap',
 'ambient pop',
 'downtempo',
 'electronica',
 'trip hop',
 'afrobeat',
 'electronic

**Next we need to find the most common occuring genres to base our recommendations on**

In [5]:
from collections import Counter
most_common_genres = [genre for genre, genre_count in Counter(genre_list).most_common(8)]
print(most_common_genres)

['soul', 'funk', 'indie soul', 'motown', 'quiet storm', 'classic soul', 'southern soul', 'indie jazz']


To get ideas for songs to recommend, the search end point and recommendations end point in the API can be used. For the search end point, spaces will not work, so it is best to use the "replace" method to change spaces to "%20"

The recommendations end point in the Spotify API can also be used to generate a list of song recommendations, this does not require any text to be replaced. 

In [6]:
search_genres = []
for genre in most_common_genres:
    search = genre.replace(" ", "%20")
    search_genres.append(search)
print(search_genres)

['soul', 'funk', 'indie%20soul', 'motown', 'quiet%20storm', 'classic%20soul', 'southern%20soul', 'indie%20jazz']


#### Now find recommendations using the 5 most common genres. 
It is worth noting that the recommendations end point has lots of different paramaters and kwargs, and bespoke recommendations can be made using the detailed spotify audio features. For now though, we will just use genre.

Firstly we need to get all the available genre seeds used to generate recommendations

In [7]:
seeds = sp.recommendation_genre_seeds()
print(seeds['genres'])

['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient', 'anime', 'black-metal', 'bluegrass', 'blues', 'bossanova', 'brazil', 'breakbeat', 'british', 'cantopop', 'chicago-house', 'children', 'chill', 'classical', 'club', 'comedy', 'country', 'dance', 'dancehall', 'death-metal', 'deep-house', 'detroit-techno', 'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm', 'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk', 'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove', 'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle', 'heavy-metal', 'hip-hop', 'holidays', 'honky-tonk', 'house', 'idm', 'indian', 'indie', 'indie-pop', 'industrial', 'iranian', 'j-dance', 'j-idol', 'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino', 'malay', 'mandopop', 'metal', 'metal-misc', 'metalcore', 'minimal-techno', 'movies', 'mpb', 'new-age', 'new-release', 'opera', 'pagode', 'party', 'philippines-opm', 'piano', 'pop', 'pop-film', 'post-dubstep', 'power-po

Now we find the genres in our playlist that can be used as a genre seed

In [8]:
list_A = most_common_genres
list_B = seeds['genres']
jVal = "|".join(list_A)

matches = [i for i in list_B if i in jVal ]
truncated_matches = matches[:-2]
print(matches)

['funk', 'indie', 'jazz', 'soul']


Since there are only two matching genres and the maximum seeds are 5, we can add 3 artists as a recommendation seeds. Lets find the most common artists by firstly making a list of the artists in the playlist.

In [9]:
playlist_songs = test_playlist_col.find({})
artist_list = []
for song in playlist_songs:
    artist_list.extend(song['artist_id'])
artist_list

['5m8H6zSadhu1j9Yi04VLqD',
 '1ThoqLcyIYvZn7iWbj8fsj',
 '1ThoqLcyIYvZn7iWbj8fsj',
 '1ThoqLcyIYvZn7iWbj8fsj',
 '5fbhwqYYh4YwUoEs582mq5',
 '0epOFNiUfyON9EYx7Tpr6V',
 '59QKG3eYCCmqC6SqUwXvSw',
 '6J7biCazzYhU3gM9j1wfid',
 '3oKRxpszQKUjjaHz388fVA',
 '3zmfs9cQwzJl575W1ZYXeT',
 '1uiEZYehlNivdK3iQyAbye',
 '2rspptKP0lPBdlJJAJHqht',
 '0Y4inQK6OespitzD6ijMwb',
 '24icoQNJSEWNu3XvqKBR68',
 '1itM5tXaK5THggpXA7ovAe',
 '73sIBHcqh3Z3NyqHKZ7FOL',
 '1P6U1dCeHxPui5pIrGmndZ',
 '0cLrgpG3pW4MX4nD8h6qCc',
 '2mVVjNmdjXZZDvhgQWiakk',
 '7tr9pbgNEKtG0GQTKe08Tz',
 '4YHtIE7FI8ITfekzzN5Jpl']

In [11]:
most_common_artists = [artist for artist, artist_count in Counter(artist_list).most_common(2)]
print(most_common_artists)

['1ThoqLcyIYvZn7iWbj8fsj', '5m8H6zSadhu1j9Yi04VLqD']


Now we have 3 artist seeds and 2 genre seeds, we can build or query parameters

In [12]:
recommended_tracks = sp.recommendations(seed_genres = truncated_matches, seed_artists = most_common_artists, limit=20)
print(recommended_tracks['tracks'])

[{'album': {'album_type': 'ALBUM', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0SwO7SWeDHJijQ3XNS7xEE'}, 'href': 'https://api.spotify.com/v1/artists/0SwO7SWeDHJijQ3XNS7xEE', 'id': '0SwO7SWeDHJijQ3XNS7xEE', 'name': 'MGMT', 'type': 'artist', 'uri': 'spotify:artist:0SwO7SWeDHJijQ3XNS7xEE'}], 'available_markets': ['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JO', 'JP', 'KW', 'LB', 'LI', 'LT', 'LU', 'LV', 'MA', 'MC', 'MT', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'OM', 'PA', 'PE', 'PH', 'PL', 'PS', 'PT', 'PY', 'QA', 'RO', 'SA', 'SE', 'SG', 'SK', 'SV', 'TH', 'TN', 'TR', 'TW', 'US', 'UY', 'VN', 'ZA'], 'external_urls': {'spotify': 'https://open.spotify.com/album/3HA1Ru1gEAgaxTywkJmBOL'}, 'href': 'https://api.spotify.com/v1/albums/3HA1Ru1gEAgaxTywkJmBOL', 'id': '3HA1Ru1gEAgaxTywkJmB

Here are the recommended tracks from Spotify, now we need to make sure there are no duplicate tracks that exist in the existing playlist, then we need to get the audio features for each track and prepare them for our classifier. 

In [13]:
for track  in recommended_tracks['tracks']:
    if any(track['id'] in songs for songs in playlist_songs):
        print('match found')
    else:
        print('no match')

no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match


No matches have been found, now we can begin getting the audio features for each song and building a data frame for our model to make predictions on. 

In [14]:
dataframe_columns = ['track', 'track_id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
song_list = []
for track in recommended_tracks['tracks']:
    features = sp.audio_features(track['id'])
    if (features[0] is not None):
                song_record = { 
                    "track": track['name'], 
                    "track_id": track['id'],
                    "danceability" : features[0]['danceability'],
                    "energy" : features[0]['energy'],
                    "key" : features[0]['key'],
                    "loudness" : features[0]['loudness'],
                    "mode" : features[0]['mode'],
                    "speechiness" : features[0]['speechiness'],
                    "acousticness" : features[0]['acousticness'],
                    "instrumentalness" : features[0]['instrumentalness'],
                    "liveness" : features[0]['liveness'],
                    "valence" : features[0]['valence'],
                    "tempo" : features[0]['tempo'],
                    "duration_ms" : features[0]['duration_ms'],
                    "time_signature" : features[0]['time_signature']
                } 
                song_list.append(song_record)
#print(song_list)
dfRecommendations = pd.DataFrame.from_dict(song_list)
dfRecommendations

,track,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Congratulations,6kblAEj0T0312fv5QWsXzo,0.761,0.637,7,-8.169,1,0.0268,0.4390,0.008200,0.1020,0.656,125.115,235387,4
1,Can We Pretend,0S5ZoKKZD3bllkvhEoPimZ,0.596,0.313,9,-13.487,1,0.0440,0.8530,0.201000,0.0601,0.489,82.067,229573,4
2,Uptight (Everything's Alright),5q5q7yDHiVkBnaivsda4TN,0.610,0.653,5,-9.956,0,0.0291,0.0287,0.000000,0.1350,0.833,133.403,175067,4
3,Made Up In Blue,1kehzYn3v8Y16rR3UDGzbE,0.461,0.778,2,-10.950,0,0.0695,0.5900,0.444000,0.4600,0.858,143.312,242427,4
4,What Is Hip?,66TSoVnJ1P9Tyok03zUlTr,0.666,0.733,9,-5.884,1,0.0563,0.1380,0.000349,0.0892,0.597,101.858,304733,4
5,What Was I Thinkin' In My head,6mckDm4xaEdyBF03nQskp9,0.840,0.888,1,-10.681,1,0.0370,0.1120,0.000005,0.4010,0.909,110.548,240720,4
6,Oh Woman Oh Man,4fYG7YvImWlUuOZVuvV6Ic,0.475,0.510,8,-6.563,1,0.0274,0.7190,0.000010,0.1120,0.226,79.900,277613,4
7,Better off Dead,2MwjZo81ldYYAah74RNQ85,0.627,0.733,5,-9.803,1,0.0409,0.4630,0.000033,0.0925,0.822,90.506,134480,4
8,Celebration - Single Version,6CSLNGruNhqpb5zhfs5n3i,0.843,0.656,1,-11.184,1,0.0595,0.0466,0.018700,0.1690,0.931,121.112,215653,4
9,No Coffee,7hzroYwyyldxIkiw3AFCrl,0.561,0.520,7,-7.284,1,0.0628,0.3440,0.000000,0.1060,0.668,199.854,246893,4


Now we have a data frame which we can use to predict if a song is gym suitable or not. Before this happens, we need to train our model. We will do this with 10 thousands records of training data using the random forest algorthim. 

In [15]:
training_col = db['medium_test_songlist']
training_songs = training_col.find({})
dfTraining =  pd.DataFrame(list(training_songs))
dfTraining

,_id,track,track_id,artist_name,artist_id,from_playlist,genres,danceability,energy,key,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,workout_suitable
0,5e7e4fa0b96e22bd0ee61d99,'Till I Collapse,4xkOaSrkexMciUUogZKVTS,"[Eminem, Nate Dogg]","[7dGJo4pcD2V6oG8kP0tJRR, 1Oa0bMld0A3u5OTYfMzp5h]","Gym Motivation : Workout Motivation, Training ...","[detroit hip hop, g funk, hip hop, rap, g funk...",0.548,0.8470,1,...,1,0.1860,0.06220,0.000000,0.0816,0.1000,171.447,297787,4,1
1,5e7e4fa1b96e22bd0ee61d9a,Numb / Encore,5sNESr6pQfIhL3krM8CtZn,"[JAY-Z, Linkin Park]","[3nFkdlSjzX9mRTtwJOzDYB, 6XyY86QOPPrYVGvF9ch6wz]","Gym Motivation : Workout Motivation, Training ...","[east coast hip hop, hip hop, pop rap, rap, al...",0.687,0.7930,2,...,1,0.1660,0.06030,0.000000,0.5820,0.7510,107.045,205733,4,1
2,5e7e4fa1b96e22bd0ee61d9b,Remember the Name (feat. Styles of Beyond),6ndmKwWqMozN2tcZqzCX4K,"[Fort Minor, Styles Of Beyond]","[7dWYWUbO68rXJOcyA7SpJk, 5bf6yYgHODBW5EreBZshpX]","Gym Motivation : Workout Motivation, Training ...","[rap rock, cali rap, rap rock]",0.688,0.8350,8,...,1,0.0911,0.05830,0.000003,0.0795,0.8800,84.858,230493,4,1
3,5e7e4fa1b96e22bd0ee61d9c,In Da Club,4RY96Asd9IefaL3X4LOLZ8,[50 Cent],[3q7HBObVc0L8jNeTe5Gofh],"Gym Motivation : Workout Motivation, Training ...","[east coast hip hop, gangster rap, hip hop, po...",0.902,0.7200,6,...,0,0.3470,0.26000,0.000000,0.0749,0.8050,90.059,193467,4,1
4,5e7e4fa1b96e22bd0ee61d9d,Stronger,4fzsfWzRhPawzqhX8Qt9F3,[Kanye West],[5K4W6rqBFWDnAN6FQUkS6x],"Gym Motivation : Workout Motivation, Training ...","[chicago rap, rap]",0.617,0.7170,10,...,0,0.1530,0.00564,0.000000,0.4080,0.4900,103.992,311867,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12678,5e7e63f38aa9abc22c18141a,Hallelujah,0uCGsNZqjHgiYO4BwN6Cjw,[Ben Laver],[3lTGvG2QAIoGMp7BKeH4C0],Calming Instrumental Covers,[],0.428,0.0415,7,...,0,0.0476,0.98500,0.942000,0.1060,0.2810,71.353,165882,4,0
12679,5e7e63f48aa9abc22c18141b,Close To Me,0kPLGFA5BjDGVkDbu4bLzE,[Mia Carrera],[00zoOzjXViIF4Fs8XQSK9m],Calming Instrumental Covers,"[calming instrumental, piano cover]",0.636,0.2410,4,...,1,0.0494,0.99000,0.902000,0.1510,0.1210,102.094,203587,4,0
12680,5e7e63f48aa9abc22c18141c,Love Me Like You Do,1nyxHNRjeyTlJQwsLMH7C0,[Anna Kitkowska],[45Miu8OyhYvkkQBXhn0MfQ],Calming Instrumental Covers,"[calming instrumental, chill guitar]",0.635,0.2310,11,...,0,0.0419,0.93900,0.947000,0.0925,0.3210,144.229,198208,4,0
12681,5e7e63f48aa9abc22c18141d,Fix You,2ZzqC4NSi374IufPYvGMP6,[Music Lab Collective],[1ylcY77FWeSVQKh5et1VGp],Calming Instrumental Covers,[classify],0.667,0.0498,3,...,1,0.0673,0.99100,0.890000,0.0818,0.0436,125.070,332427,4,0


Here is the dataframe we will use to train our classifier, next we define the features we want the classifier to learn and the feature we want to predict. These are named X and y respectively.

In [16]:
feature_cols = ['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness', 'mode', 'speechiness', 'tempo', 'time_signature']
X = dfTraining.loc[:, feature_cols]
y = dfTraining.workout_suitable

In [17]:
X

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,mode,speechiness,tempo,time_signature
0,0.06220,0.548,297787,0.8470,0.000000,1,0.0816,1,0.1860,171.447,4
1,0.06030,0.687,205733,0.7930,0.000000,2,0.5820,1,0.1660,107.045,4
2,0.05830,0.688,230493,0.8350,0.000003,8,0.0795,1,0.0911,84.858,4
3,0.26000,0.902,193467,0.7200,0.000000,6,0.0749,0,0.3470,90.059,4
4,0.00564,0.617,311867,0.7170,0.000000,10,0.4080,0,0.1530,103.992,4
...,...,...,...,...,...,...,...,...,...,...,...
12678,0.98500,0.428,165882,0.0415,0.942000,7,0.1060,0,0.0476,71.353,4
12679,0.99000,0.636,203587,0.2410,0.902000,4,0.1510,1,0.0494,102.094,4
12680,0.93900,0.635,198208,0.2310,0.947000,11,0.0925,0,0.0419,144.229,4
12681,0.99100,0.667,332427,0.0498,0.890000,3,0.0818,1,0.0673,125.070,4


In [18]:
y

0        1
1        1
2        1
3        1
4        1
        ..
12678    0
12679    0
12680    0
12681    0
12682    0
Name: workout_suitable, Length: 12683, dtype: int64

## Prediction

#### Constructing a prediction model using X and y

In [19]:
rf = RandomForestClassifier(n_estimators=40)
workout_classifier = rf.fit(X, y)
workout_classifier

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=40,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

#### Now we prepare the song recommendations to be tested 

In [20]:
X_predict = dfRecommendations.loc[:, feature_cols]
X_predict.shape

(20, 11)

### Using the classifer to predict the recommendations (X_predict)

In [21]:
new_prediction_class = workout_classifier.predict(X_predict)
new_prediction_class

array([1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1])

 This is the array of predictions, we can build a new dataframe to display this information more clearly

In [22]:
dfClassifiedRecommendations = pd.DataFrame({
    'Track Name': dfRecommendations.track,
    'Track_id': dfRecommendations.track_id,
    'Prediction': new_prediction_class
})
dfClassifiedRecommendations

,Track Name,Track_id,Prediction
0,Congratulations,6kblAEj0T0312fv5QWsXzo,1
1,Can We Pretend,0S5ZoKKZD3bllkvhEoPimZ,0
2,Uptight (Everything's Alright),5q5q7yDHiVkBnaivsda4TN,1
3,Made Up In Blue,1kehzYn3v8Y16rR3UDGzbE,1
4,What Is Hip?,66TSoVnJ1P9Tyok03zUlTr,1
5,What Was I Thinkin' In My head,6mckDm4xaEdyBF03nQskp9,1
6,Oh Woman Oh Man,4fYG7YvImWlUuOZVuvV6Ic,0
7,Better off Dead,2MwjZo81ldYYAah74RNQ85,1
8,Celebration - Single Version,6CSLNGruNhqpb5zhfs5n3i,1
9,No Coffee,7hzroYwyyldxIkiw3AFCrl,0


Here we have a list of songs with the classifiers prediction of if it is suitable for working out to or not. 
Now all that is left to do is display only the songs predicted as suitable for our hypothetical user.

In [23]:
dict1 = dfClassifiedRecommendations.to_dict('records')
for track in dict1:
    print(track)

{'Track Name': 'Congratulations', 'Track_id': '6kblAEj0T0312fv5QWsXzo', 'Prediction': 1}
{'Track Name': 'Can We Pretend', 'Track_id': '0S5ZoKKZD3bllkvhEoPimZ', 'Prediction': 0}
{'Track Name': "Uptight (Everything's Alright)", 'Track_id': '5q5q7yDHiVkBnaivsda4TN', 'Prediction': 1}
{'Track Name': 'Made Up In Blue', 'Track_id': '1kehzYn3v8Y16rR3UDGzbE', 'Prediction': 1}
{'Track Name': 'What Is Hip?', 'Track_id': '66TSoVnJ1P9Tyok03zUlTr', 'Prediction': 1}
{'Track Name': "What Was I Thinkin' In My head", 'Track_id': '6mckDm4xaEdyBF03nQskp9', 'Prediction': 1}
{'Track Name': 'Oh Woman Oh Man', 'Track_id': '4fYG7YvImWlUuOZVuvV6Ic', 'Prediction': 0}
{'Track Name': 'Better off Dead', 'Track_id': '2MwjZo81ldYYAah74RNQ85', 'Prediction': 1}
{'Track Name': 'Celebration - Single Version', 'Track_id': '6CSLNGruNhqpb5zhfs5n3i', 'Prediction': 1}
{'Track Name': 'No Coffee', 'Track_id': '7hzroYwyyldxIkiw3AFCrl', 'Prediction': 0}
{'Track Name': "It's A Shame", 'Track_id': '1l1YTy9nJ0trwhsCGcimly', 'Predict

In [24]:
dictRecommend = []
for track in dict1: 
    if(track['Prediction'] == 1):
        dictRecommend.append(track)
for track in dictRecommend:
    print(track)

{'Track Name': 'Congratulations', 'Track_id': '6kblAEj0T0312fv5QWsXzo', 'Prediction': 1}
{'Track Name': "Uptight (Everything's Alright)", 'Track_id': '5q5q7yDHiVkBnaivsda4TN', 'Prediction': 1}
{'Track Name': 'Made Up In Blue', 'Track_id': '1kehzYn3v8Y16rR3UDGzbE', 'Prediction': 1}
{'Track Name': 'What Is Hip?', 'Track_id': '66TSoVnJ1P9Tyok03zUlTr', 'Prediction': 1}
{'Track Name': "What Was I Thinkin' In My head", 'Track_id': '6mckDm4xaEdyBF03nQskp9', 'Prediction': 1}
{'Track Name': 'Better off Dead', 'Track_id': '2MwjZo81ldYYAah74RNQ85', 'Prediction': 1}
{'Track Name': 'Celebration - Single Version', 'Track_id': '6CSLNGruNhqpb5zhfs5n3i', 'Prediction': 1}
{'Track Name': "It's A Shame", 'Track_id': '1l1YTy9nJ0trwhsCGcimly', 'Prediction': 1}
{'Track Name': "Luv N' Haight - Single Version", 'Track_id': '4Phj7Zce9seBEAaqiyK12r', 'Prediction': 1}
{'Track Name': 'Flashlight', 'Track_id': '6VSm4HuSiOGWjbt6zpPnvh', 'Prediction': 1}
{'Track Name': 'Feeling Lonely', 'Track_id': '5uIRujGRZv5t4fGKk

## The recommendations are:

In [25]:
dfResults = pd.DataFrame.from_dict(dictRecommend)
dfResults

,Track Name,Track_id,Prediction
0,Congratulations,6kblAEj0T0312fv5QWsXzo,1
1,Uptight (Everything's Alright),5q5q7yDHiVkBnaivsda4TN,1
2,Made Up In Blue,1kehzYn3v8Y16rR3UDGzbE,1
3,What Is Hip?,66TSoVnJ1P9Tyok03zUlTr,1
4,What Was I Thinkin' In My head,6mckDm4xaEdyBF03nQskp9,1
5,Better off Dead,2MwjZo81ldYYAah74RNQ85,1
6,Celebration - Single Version,6CSLNGruNhqpb5zhfs5n3i,1
7,It's A Shame,1l1YTy9nJ0trwhsCGcimly,1
8,Luv N' Haight - Single Version,4Phj7Zce9seBEAaqiyK12r,1
9,Flashlight,6VSm4HuSiOGWjbt6zpPnvh,1
